In [187]:
%%capture
!pip install easydict
!pip install torch
!pip install transformers

In [250]:
#import necessary modules
import pandas as pd
import numpy as np
from functools import reduce
import random
import torch
import transformers
from torch import nn, optim
import torch.nn.functional as F
from tensorflow.python.eager import context
from tensorflow.python.framework import config as tf_config
from tensorflow.python.framework import ops
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
from easydict import EasyDict as edict
import transformers
import torch
from transformers import AutoModel, AutoTokenizer, BertTokenizer
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, Adafactor, T5ForConditionalGeneration, T5Tokenizer
from transformers import get_linear_schedule_with_warmup
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [251]:
df = pd.read_csv('final_texts.csv', sep='\t', index_col=0)
df.head()

,text
0,DEĞİŞEN EVRENSEL BİR İKON Marilyn...
1,Ben ölüm... Herkes korkar benden. Herkes tar...
2,GÜZELLİK Mİ ÇİRKİNLİK Mİ HAVUZ BAŞINDA MANKENL...
3,Hayatım boyunca evrende sadece Dünya’da yaşam ...
4,"Bir Şehrin İki Yakasında ""İki büyük cihanın k..."


In [252]:
df['lowercase'] = df['text'].str.lower()

In [253]:
df.head()

,text,lowercase
0,DEĞİŞEN EVRENSEL BİR İKON Marilyn...,deği̇şen evrensel bi̇r i̇kon mari...
1,Ben ölüm... Herkes korkar benden. Herkes tar...,ben ölüm... herkes korkar benden. herkes tar...
2,GÜZELLİK Mİ ÇİRKİNLİK Mİ HAVUZ BAŞINDA MANKENL...,güzelli̇k mi̇ çi̇rki̇nli̇k mi̇ havuz başinda m...
3,Hayatım boyunca evrende sadece Dünya’da yaşam ...,hayatım boyunca evrende sadece dünya’da yaşam ...
4,"Bir Şehrin İki Yakasında ""İki büyük cihanın k...","bir şehrin i̇ki yakasında ""i̇ki büyük cihanın..."


In [254]:
whole_text = reduce(lambda x,y: x+'\n'+y, df['lowercase'].values)

In [193]:
whole_text[:100]

'deği̇şen evrensel bi̇r i̇kon              marilyn monroe akıllarda hep güzel sarışın olarak kaldı. m'

In [217]:
chars = tuple(set(whole_text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

In [218]:
print(list(int2char.items())[:15])

[(0, '!'), (1, '—'), (2, 'δ'), (3, '"'), (4, '‐'), (5, '<'), (6, '/'), (7, '‑'), (8, '̆'), (9, 'p'), (10, '…'), (11, 'x'), (12, 'ứ'), (13, 'ï'), (14, 'ş')]


In [219]:
print(list(char2int.items())[:15])

[('!', 0), ('—', 1), ('δ', 2), ('"', 3), ('‐', 4), ('<', 5), ('/', 6), ('‑', 7), ('̆', 8), ('p', 9), ('…', 10), ('x', 11), ('ứ', 12), ('ï', 13), ('ş', 14)]


In [220]:
encoded = np.array([char2int[ch] for ch in whole_text])
encoded[:50]

array([ 98,  34,  25,  69,  43,  14,  34, 117,  79,  34,  46, 157,  34,
       117, 152,  34,  18,  79, 144,  69,  43, 157,  79,  69,  43, 149,
        52, 117,  79,  79,  79,  79,  79,  79,  79,  79,  79,  79,  79,
        79,  79,  79,  56,  90, 157,  69,  18,  93, 117,  79])

In [221]:
print(f'Text_Length:{len(whole_text)} - Token Count: {len(char2int)}')

Text_Length:30417478 - Token Count: 169


In [222]:
def get_batches(arr, batch_size, seq_length):
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [223]:
next(get_batches(encoded, 5,10))

(array([[ 98,  34,  25,  69,  43,  14,  34, 117,  79,  34],
        [151,  69, 151,  79, 144,  69, 157,  79,  34, 152],
        [144,  69, 157,  79,  79,  93,  90, 158,  95, 152],
        [  9,  89,  79,  79, 109,  79,  52,  18,  56,  90],
        [ 34,  46, 157,  69,  56,  59,  79, 149, 132,  14]]),
 array([[ 34,  25,  69,  43,  14,  34, 117,  79,  34,  46],
        [ 69, 151,  79, 144,  69, 157,  79,  34, 152,  69],
        [ 69, 157,  79,  79,  93,  90, 158,  95, 152,  95],
        [ 89,  79,  79, 109,  79,  52,  18,  56,  90, 152],
        [ 46, 157,  69,  56,  59,  79, 149, 132,  14,  90]]))

In [224]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [225]:
conf = edict()
conf.seed_val = 1903
conf.batch_size = 16
conf.seq_length = 100
conf.use_gpu = True
conf.vocab_size = len(int2char)
conf.weight_decay = 0.001
conf.learning_rate = 5e-4
conf.warmup_steps = 20000
conf.epochs = 20
conf.gradient_clip = 2

In [226]:
random.seed(conf.seed_val)
np.random.seed(conf.seed_val)
torch.manual_seed(conf.seed_val)
torch.cuda.manual_seed_all(conf.seed_val)

In [227]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [240]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=500):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
        
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [241]:
# define and print the net
n_hidden=256
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(169, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=256, out_features=169, bias=True)
)


In [242]:
len(encoded)/12800

2376.36546875

In [244]:
batch_size = 128
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior
train_on_gpu = True

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=500)

Epoch: 1/20... Step: 500... Loss: 2.2443... Val Loss: 2.1491
Epoch: 1/20... Step: 1000... Loss: 1.9635... Val Loss: 1.8432
Epoch: 1/20... Step: 1500... Loss: 1.8115... Val Loss: 1.6869
Epoch: 1/20... Step: 2000... Loss: 1.7238... Val Loss: 1.5896
Epoch: 2/20... Step: 2500... Loss: 1.6250... Val Loss: 1.5348
Epoch: 2/20... Step: 3000... Loss: 1.5975... Val Loss: 1.4941
Epoch: 2/20... Step: 3500... Loss: 1.5545... Val Loss: 1.4649
Epoch: 2/20... Step: 4000... Loss: 1.5542... Val Loss: 1.4434
Epoch: 3/20... Step: 4500... Loss: 1.5001... Val Loss: 1.4246
Epoch: 3/20... Step: 5000... Loss: 1.5340... Val Loss: 1.4098
Epoch: 3/20... Step: 5500... Loss: 1.5233... Val Loss: 1.3956
Epoch: 3/20... Step: 6000... Loss: 1.4912... Val Loss: 1.3857
Epoch: 4/20... Step: 6500... Loss: 1.5105... Val Loss: 1.3758
Epoch: 4/20... Step: 7000... Loss: 1.5279... Val Loss: 1.3677
Epoch: 4/20... Step: 7500... Loss: 1.4596... Val Loss: 1.3583
Epoch: 4/20... Step: 8000... Loss: 1.5100... Val Loss: 1.3521
Epoch: 4/

In [245]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [246]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [248]:
print(sample(net, 1000, prime='bir sabah erkenden kalkıp aydos ormanına doğru yürüdüm.', top_k=5))

bir sabah erkenden kalkıp aydos ormanına doğru yürüdüm.      konuşmayı birbirimizi kalmadıkları anlarına koymayacağımız bir duygudur. ama bu kızın  kendi içinin bağından bağımsız olması gerekiyor bile kendimizi başkaları gibi bir şeyler var  bence. hayatınızda başarılı olduğunuz bir şekilde bir savaş yapar ve daha iyi bu da  kurulması gereken anılardan sıkılmaya başlamıştır. karşılık veriyorsunuz. kimsede  de babamın doğuyor, belki de aslında bunu bildiği kişiyi daha da doğal bırakmayacak.  hiçbir şeyi yapmış olursanız aslında kendi içinizdekiliklerin dinlenmiştir.  kendimizin yanan sorunlarla kaybolacak kadar sanki o zamanların da bile doğru olması.    hepimiz asıl değerlerimizi sevdiğiniz kadarının bir aranızda bir şekilde bir karar vermesini  değil miydi ya da bu dünyada da başımıza bu kadar durum yarının bir süreçte olmayan  kabul edilmesi de kim bu yanlış düşünceleri başlar. ancak bu dünyanın bir bakış açısına  giriyoruz.   karşı altın boyunca bekleyen bir yere kadar arayacağımızı